<a href="https://www.kaggle.com/code/raviiloveyou/graphneurons-predicting-tensorflow?scriptVersionId=157052729" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predict-ai-model-runtime/sample_submission.csv
/kaggle/input/predict-ai-model-runtime/npz_all/npz/tile/xla/valid/resnet_v1_50_official_batch_128_bf16_2bea628b72fd4080.npz
/kaggle/input/predict-ai-model-runtime/npz_all/npz/tile/xla/test/04ae9238c653f8ae08f60f2c03615f0b.npz
/kaggle/input/predict-ai-model-runtime/npz_all/npz/tile/xla/train/retinanet.4x4.fp32_-431a58cc30e72ec6.npz
/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/nlp/random/valid/small_bert_bert_en_uncased_L-6_H-512_A-8_batch_size_64_test.npz
/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/nlp/random/test/b2fdde3b72980907578648774101543e.npz
/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/nlp/random/train/small_bert_bert_en_uncased_L-2_H-768_A-12_batch_size_16_test.npz
/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/nlp/default/valid/small_bert_bert_en_uncased_L-6_H-512_A-8_batch_size_64_test.npz
/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout/nlp/default/test/b

In [2]:
!pip install torch_geometric

  Obtaining dependency information for torch_geometric from https://files.pythonhosted.org/packages/65/4e/6f9a75548a93fedcd4514ae2de9bee1e91bade6b73252b4da32f0e42ac52/torch_geometric-2.4.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.8 MB/s eta 0:00:00


In [3]:
!pip install tensorflow_gnn --pre
!pip install tensorflow_ranking

  Obtaining dependency information for tensorflow_gnn from https://files.pythonhosted.org/packages/54/c2/4deb5d89417c5fec06a34cd0544b5be74c69680134378becafd66faf0283/tensorflow_gnn-1.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for google-vizier>=0.0.13 from https://files.pythonhosted.org/packages/94/0d/3fecfa27998d7161560cb0de6b6da0681328bbd3791bfd0244795b134f72/google_vizier-0.1.13-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Obtaining dependency information for portpicker>=1.3.1 from https://files.pythonhosted.org/packages/32/2d/440e4d7041fff89f28f483733eb617127aa866135c2dc719e05893f089e1/portpicker-1.6.0-py3-none-any.whl.metadata
  Obtaining dependency information for grpcio-tools>=1.35.0 from https://files.pythonhosted.org/packages/3c/7d/00a156dba65c9965e6e94988ab518c4ea88f95e1b70c2b61b34dd65124b5/grpcio_tools-1.60.0-cp310-cp310-manylinux_2_17_x86_64.many

In [4]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch import Tensor
from torch_geometric.nn import GCNConv,SAGEConv
from torch_geometric.datasets import Planetoid
from torch.utils.data import DataLoader,Dataset
from timm.scheduler import CosineLRScheduler
import matplotlib.pyplot as plt
device="cuda" if torch.cuda.is_available() else "cpu"
import tensorflow as tf
import tensorflow_gnn as tfgnn
import tensorflow_ranking as tfr

In [5]:
import tpugraphsv1_layout_data_py as layout_data
import tpugraphsv1_tile_data_py as title_data
import tpugraphsv1_implicit_py as implicit

In [6]:
def _mlp(dims, hidden_activation, l2reg=1e-4, use_bias=True):
  """Helper function for multi-layer perceptron (MLP)."""
  layers = []
  for i, dim in enumerate(dims):
    if i > 0:
      layers.append(tf.keras.layers.Activation(hidden_activation))
    layers.append(tf.keras.layers.Dense(
        dim, kernel_regularizer=tf.keras.regularizers.l2(l2reg),
        use_bias=use_bias))
  return tf.keras.Sequential(layers)


class _OpEmbedding(tf.keras.Model):
  """Embeds GraphTensor.node_sets['op']['op'] nodes into feature 'op_e'."""

  def __init__(self, num_ops: int, embed_d: int, l2reg: float = 1e-4):
    super().__init__()
    self.embedding_layer = tf.keras.layers.Embedding(
        num_ops, embed_d, activity_regularizer=tf.keras.regularizers.l2(l2reg))

  def call(
      self, graph: tfgnn.GraphTensor,
      training: bool = False) -> tfgnn.GraphTensor:
    op_features = dict(graph.node_sets['op'].features)
    op_features['op_e'] = self.embedding_layer(
        tf.cast(graph.node_sets['op']['op'], tf.int32))
    return graph.replace_features(node_sets={'op': op_features})

In [7]:
LAYOUT_DATA_ROOT = '/kaggle/input/predict-ai-model-runtime/npz_all/npz/layout'
SOURCE = 'xla'  # Can be "xla" or "nlp"
SEARCH = 'random'  # Can be "random" or "default"

# Batch size information.
BATCH_SIZE = 16  # Number of graphs per batch.
CONFIGS_PER_GRAPH = 5  # Number of configurations (features and target values) per graph.
MAX_KEEP_NODES = 1000

In [8]:
import os
layout_data_root_dir = os.path.join(
      os.path.expanduser(LAYOUT_DATA_ROOT), SOURCE, SEARCH)

layout_npz_dataset = layout_data.get_npz_dataset(
    layout_data_root_dir,
    min_train_configs=CONFIGS_PER_GRAPH,
    max_train_configs=500,  # If any graph has more than this configurations, it will be filtered [speeds up loading + training]
    cache_dir='cache'
)

dataset cache file:  cache/317e146d60640edb255d1dcb2d3235c8-cache.npz


100%|██████████| 69/69 [01:08<00:00,  1.01it/s]


Saving ...
wrote cache/317e146d60640edb255d1dcb2d3235c8-cache.npz
wrote cache/317e146d60640edb255d1dcb2d3235c8-cache.npz.graphs.txt
dataset cache file:  cache/ed1b35ba5a151cd92dc0e4d41f4160ce-cache.npz


100%|██████████| 7/7 [00:07<00:00,  1.13s/it]


Saving ...
wrote cache/ed1b35ba5a151cd92dc0e4d41f4160ce-cache.npz
wrote cache/ed1b35ba5a151cd92dc0e4d41f4160ce-cache.npz.graphs.txt
dataset cache file:  cache/437403654dcd7a1b5d98d25edfb15ce6-cache.npz


100%|██████████| 8/8 [00:01<00:00,  5.29it/s]


Saving ...
wrote cache/437403654dcd7a1b5d98d25edfb15ce6-cache.npz
wrote cache/437403654dcd7a1b5d98d25edfb15ce6-cache.npz.graphs.txt


In [9]:
def pair_layout_graph_with_label(graph: tfgnn.GraphTensor):
    
    label=tf.cast(graph.node_sets["g"]["runtimes"],tf.float32)/1e7
    return graph,label



In [10]:
layout_train=(
    layout_npz_dataset.train.get_graph_tensors_dataset(
        CONFIGS_PER_GRAPH,max_nodes=MAX_KEEP_NODES)
        .shuffle(100,reshuffle_each_iteration=True)
        .batch(BATCH_SIZE,drop_remainder=False)
        .map(tfgnn.GraphTensor.merge_batch_to_components)
        .map(pair_layout_graph_with_label))
layout_valid=(
    layout_npz_dataset.validation.get_graph_tensors_dataset(
        CONFIGS_PER_GRAPH)
        .batch(BATCH_SIZE,drop_remainder=False)
        .map(tfgnn.GraphTensor.merge_batch_to_components)
        .map(pair_layout_graph_with_label))

In [11]:
graph_batch,config_runtimes=next(iter(layout_train.take(1)))
print("graph_batch=")
print(graph_batch)
print("\n\n")
print("config_runtimes=")
print(config_runtimes)

graph_batch=
GraphTensor(
  context=Context(features={}, sizes=[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], shape=(), indices_dtype=tf.int32),
  node_set_names=['op', 'nconfig', 'g'],
  edge_set_names=['config', 'feed', 'g_op', 'g_config', 'sampled_config', 'sampled_feed'])



config_runtimes=
tf.Tensor(
[[   7.790108     7.8527017    7.7896423   12.340676     7.8639336]
 [   5.050417     3.694046     5.254685     3.7243912    5.3103094]
 [  21.35933     23.25172     23.253271    13.378965    12.227056 ]
 [   7.7916303    7.7912865    7.7919903    7.791703     7.7933416]
 [1311.5782    1508.5378     495.7099    1026.7607     810.0628   ]
 [1239.5872    1230.5071    1282.7224    1296.4832    1763.7079   ]
 [  28.117546   244.07838     28.119484    34.02736    178.92744  ]
 [  98.79405     79.11045     96.45612    101.037964    79.107376 ]
 [  19.211882    19.208782    30.871626    31.256361    19.07562  ]
 [  51.394337    51.401768    51.401577    51.400196    51.404865 ]
 [   7.9626513    7.9631

In [12]:
print('graph_batch.context =', graph_batch.context)
# Note: graph_batch.context.sizes must be equal to BATCH_SIZE.
# Lets print-out all features for all nodesets.

for node_set_name in sorted(graph_batch.node_sets.keys()):
    print(f'\n\n #####  NODE SET "{node_set_name}" #########')
    print('** Has sizes: ', graph_batch.node_sets[node_set_name].sizes)
    for feature_name in graph_batch.node_sets[node_set_name].features.keys():
        print(f'\n Feature "{feature_name}" has values')
        print(graph_batch.node_sets[node_set_name][feature_name])

graph_batch.context = Context(features={}, sizes=[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], shape=(), indices_dtype=tf.int32)


 #####  NODE SET "g" #########
** Has sizes:  tf.Tensor([1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], shape=(16,), dtype=int32)

 Feature "graph_id" has values
tf.Tensor(
[b'transformer_tf2_dynamic_shape'
 b'tf2_bert_pretrain_dynamic_sequence_length' b'resnet50.4x4.bf16'
 b'mlperf_nmt_1_shard_batch_8' b'mask_rcnn_batch_4_bf16_img1408'
 b'mask_rcnn_batch_16_bf16_img1024' b'resnet_v2_50_batch_16'
 b'mlperf_nmt_batch_64' b'resnet50.2x2.fp32' b'openai_v0_rnn_natural'
 b'alexnet_train_batch_32' b'inference_mlperf_resnet_batch_256'
 b'resnet_v2_200_batch_64' b'openai_v0_rnn_optimized'
 b'transformer.4x4.bf16' b'magenta'], shape=(16,), dtype=string)

 Feature "runtimes" has values
tf.Tensor(
[[   77901081    78527014    77896424   123406759    78639338]
 [   50504167    36940461    52546847    37243911    53103092]
 [  213593290   232517198   232532700   133789658   122270554]
 [   779

In [13]:
print('\n config edge set: ', graph_batch.edge_sets['config'])  
print('\n config source nodes: ', graph_batch.edge_sets['config'].adjacency.source)
print('\n config target nodes: ', graph_batch.edge_sets['config'].adjacency.target)
print('\n g_op edge set: ', graph_batch.edge_sets['g_op'])
print('\n g_config edge set: ', graph_batch.edge_sets['g_config'])



 config edge set:  EdgeSet(features={}, sizes=[ 921 1206  161 1253  583  634  162 1254  161   72   26   55  612   62
 1770  908], adjacency=Adjacency(source=('nconfig', <tf.Tensor: shape=(9840,), dtype=tf.int32>), target=('op', <tf.Tensor: shape=(9840,), dtype=tf.int32>)))

 config source nodes:  tf.Tensor([   0    1    2 ... 9837 9838 9839], shape=(9840,), dtype=int32)

 config target nodes:  tf.Tensor([  2878   2881   2886 ... 172954 173161 173207], shape=(9840,), dtype=int32)

 g_op edge set:  EdgeSet(features={}, sizes=[14680 22385  5605 15642 12565 13342  5162 15809  5279  1383   372  1111
 19662  1277 26906 12062], adjacency=Adjacency(source=('g', <tf.Tensor: shape=(173242,), dtype=tf.int32>), target=('op', <tf.Tensor: shape=(173242,), dtype=tf.int32>)))

 g_config edge set:  EdgeSet(features={}, sizes=[ 921 1206  161 1253  583  634  162 1254  161   72   26   55  612   62
 1770  908], adjacency=Adjacency(source=('g', <tf.Tensor: shape=(9840,), dtype=tf.int32>), target=('nconfig'

In [14]:
print(graph_batch.edge_sets['config'])
print(graph_batch.edge_sets['config'].adjacency.source)
print(graph_batch.edge_sets['config'].adjacency.target)

EdgeSet(features={}, sizes=[ 921 1206  161 1253  583  634  162 1254  161   72   26   55  612   62
 1770  908], adjacency=Adjacency(source=('nconfig', <tf.Tensor: shape=(9840,), dtype=tf.int32>), target=('op', <tf.Tensor: shape=(9840,), dtype=tf.int32>)))
tf.Tensor([   0    1    2 ... 9837 9838 9839], shape=(9840,), dtype=int32)
tf.Tensor([  2878   2881   2886 ... 172954 173161 173207], shape=(9840,), dtype=int32)


In [15]:
ops=layout_npz_dataset.num_ops
print(f"number of ops in the dataset:",{ops})
embedding_layer = _OpEmbedding(ops,16)
graph_batch_embedded_ops=embedding_layer(graph_batch)
print(f"\n\n Before embedding,node-set op=\n' ,{graph_batch.node_sets['op']}")
print(f"\n\n After embedding,node-set op=\n' ,{graph_batch_embedded_ops.node_sets['op']}")

number of ops in the dataset: {119}


 Before embedding,node-set op=
' ,NodeSet(features={'op': <tf.Tensor: shape=(173242,), dtype=tf.uint8>, 'feats': <tf.Tensor: shape=(173242, 112), dtype=tf.float32>, 'selected': <tf.Tensor: shape=(173242,), dtype=tf.bool>}, sizes=[14680 22385  5605 15642 12565 13342  5162 15809  5279  1383   372  1111
 19662  1277 26906 12062])


 After embedding,node-set op=
' ,NodeSet(features={'op': <tf.Tensor: shape=(173242,), dtype=tf.uint8>, 'feats': <tf.Tensor: shape=(173242, 112), dtype=tf.float32>, 'selected': <tf.Tensor: shape=(173242,), dtype=tf.bool>, 'op_e': <tf.Tensor: shape=(173242, 16), dtype=tf.float32>}, sizes=[14680 22385  5605 15642 12565 13342  5162 15809  5279  1383   372  1111
 19662  1277 26906 12062])


In [16]:
op_e=graph_batch_embedded_ops.node_sets['op']['op_e']
config_features=graph_batch_embedded_ops.node_sets['nconfig']['feats']
print(f"op_e.shape ==  {op_e.shape}")
print(f"config_feature.shape == {config_features.shape}")

op_e.shape ==  (173242, 16)
config_feature.shape == (9840, 5, 14)


In [17]:
config_adj = implicit.AdjacencyMultiplier(graph_batch_embedded_ops, 'config')
print('config_adj.shape =', config_adj.shape)
resized_config_features = config_adj @ config_features
print('resized_config_features.shape =', resized_config_features.shape)

config_adj.shape = (<tf.Tensor: shape=(), dtype=int32, numpy=173242>, <tf.Tensor: shape=(), dtype=int32, numpy=9840>)
resized_config_features.shape = (173242, 5, 14)


In [18]:
class ResModel(tf.keras.Model):
    """GNN with residual connections."""

    def __init__(
        self, num_configs: int, num_ops: int, op_embed_dim: int = 32,
        num_gnns: int = 2, mlp_layers: int = 2,
        hidden_activation: str = 'leaky_relu',
        hidden_dim: int = 32, reduction: str = 'sum'):
        super().__init__()
        self._num_configs = num_configs
        self._num_ops = num_ops
        self._op_embedding = _OpEmbedding(num_ops, op_embed_dim)
        self._prenet = _mlp([hidden_dim] * mlp_layers, hidden_activation)
        self._gc_layers = []
        for _ in range(num_gnns):
            self._gc_layers.append(_mlp([hidden_dim] * mlp_layers, hidden_activation))
        self._postnet = _mlp([hidden_dim, 1], hidden_activation, use_bias=False)

    def call(self, graph: tfgnn.GraphTensor, training: bool = False):
        del training
        return self.forward(graph, self._num_configs)

    def _node_level_forward(
        self, node_features: tf.Tensor,
        config_features: tf.Tensor,
        graph: tfgnn.GraphTensor, num_configs: int,
        edgeset_prefix='') -> tf.Tensor:
        adj_op_op = implicit.AdjacencyMultiplier(
            graph, edgeset_prefix+'feed')  # op->op
        adj_config = implicit.AdjacencyMultiplier(
            graph, edgeset_prefix+'config')  # nconfig->op

        adj_op_op_hat = (adj_op_op + adj_op_op.transpose()).add_eye()
        adj_op_op_hat = adj_op_op_hat.normalize_symmetric()

        x = node_features

        x = tf.stack([x] * num_configs, axis=1)
        config_features = 100 * (adj_config @ config_features)
        x = tf.concat([config_features, x], axis=-1)
        x = self._prenet(x)
        x = tf.nn.leaky_relu(x)

        for layer in self._gc_layers:
            y = x
            y = tf.concat([config_features, y], axis=-1)
            y = tf.nn.leaky_relu(layer(adj_op_op_hat @ y))
            x += y
        return x

    def forward(
        self, graph: tfgnn.GraphTensor, num_configs: int,
        backprop=True) -> tf.Tensor:
        graph = self._op_embedding(graph)

        config_features = graph.node_sets['nconfig']['feats']
        node_features = tf.concat([
            graph.node_sets['op']['feats'],
            graph.node_sets['op']['op_e']
        ], axis=-1)

        x_full = self._node_level_forward(
            node_features=tf.stop_gradient(node_features),
            config_features=tf.stop_gradient(config_features),
            graph=graph, num_configs=num_configs)

        if backprop:
            x_backprop = self._node_level_forward(
                node_features=node_features,
                config_features=config_features,
                graph=graph, num_configs=num_configs, edgeset_prefix='sampled_')

            is_selected = graph.node_sets['op']['selected']
            # Need to expand twice as `is_selected` is a vector (num_nodes) but
            # x_{backprop, full} are 3D tensors (num_nodes, num_configs, num_feats).
            is_selected = tf.expand_dims(is_selected, axis=-1)
            is_selected = tf.expand_dims(is_selected, axis=-1)
            x = tf.where(is_selected, x_backprop, x_full)
        else:
            x = x_full

        adj_config = implicit.AdjacencyMultiplier(graph, 'config')

        # Features for configurable nodes.
        config_feats = (adj_config.transpose() @ x)

        # Global pooling
        adj_pool_op_sum = implicit.AdjacencyMultiplier(graph, 'g_op').transpose()
        adj_pool_op_mean = adj_pool_op_sum.normalize_right()
        adj_pool_config_sum = implicit.AdjacencyMultiplier(
            graph, 'g_config').transpose()
        x = self._postnet(tf.concat([
            # (A D^-1) @ Features
            adj_pool_op_mean @ x,
            # l2_normalize( A @ Features )
            tf.nn.l2_normalize(adj_pool_op_sum @ x, axis=-1),
            # l2_normalize( A @ Features )
            tf.nn.l2_normalize(adj_pool_config_sum @ config_feats, axis=-1),
        ], axis=-1))

        x = tf.squeeze(x, -1)

        return x

In [19]:
model=ResModel(CONFIGS_PER_GRAPH,layout_npz_dataset.num_ops)
loss=tfr.keras.losses.ListMLELoss()
opt=tf._optimizers.Adam(learning_rate=0.01,clipnorm=0.5)
model.compile(loss=loss,optimizer=opt,metrics=[
    tfr.keras.metrics.OPAMetric(name="opa_metric"),
])

In [20]:
early_stopping=10
best_params=None
best_val_opa=-1
best_val_at_epoch=-1
epochs=2

for i in range(epochs):
    history = model.fit(
        layout_train, epochs=1, verbose=1, validation_data=layout_valid,
        validation_freq=1)

    train_loss = history.history['loss'][-1]
    train_opa = history.history['opa_metric'][-1]
    val_loss = history.history['val_loss'][-1]
    val_opa = history.history['val_opa_metric'][-1]
    if val_opa > best_val_opa:
        best_val_opa = val_opa
        best_val_at_epoch = i
        best_params = {v.ref: v + 0 for v in model.trainable_variables}
        print(' * [@%i] Validation (NEW BEST): %s' % (i, str(val_opa)))
    elif early_stopping > 0 and i - best_val_at_epoch >= early_stopping:
      print('[@%i] Best accuracy was attained at epoch %i. Stopping.' % (i, best_val_at_epoch))
      break

# Restore best parameters.
print('Restoring parameters corresponding to the best validation OPA.')
assert best_params is not None
for v in model.trainable_variables:
    v.assign(best_params[v.ref])

5/5 [==============================] - 336s 68s/step - loss: 4.3609 - opa_metric: 0.5681 - val_loss: 4.2204 - val_opa_metric: 0.6286
 * [@0] Validation (NEW BEST): 0.6285714507102966
5/5 [==============================] - 282s 55s/step - loss: 4.1909 - opa_metric: 0.6377 - val_loss: 5.0336 - val_opa_metric: 0.4714
Restoring parameters corresponding to the best validation OPA.


In [21]:
from tensorflow.keras import layers
class CNConv(layers.Layer):
    def __init__(self, num_filters, k, activation='relu', **kwargs):
        super(CNConv, self).__init__(**kwargs)
        self.num_filters = num_filters
        self.k = k
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        self.kernel = self.add_weight("kernel",
                                      (self.k, input_shape[-1], self.num_filters))

    def cheb_polynomials(self, laplacian, k):
        cheb_polys = [tf.eye(tf.shape(laplacian)[0])]  # T_0(x) = I
        cheb_polys.append(laplacian)  # T_1(x) = L
        for i in range(2, k):
            cheb_polys.append(2 * laplacian * cheb_polys[-1] - cheb_polys[-2])
        return cheb_polys

    def call(self, inputs, laplacian):
        laplacian = tf.convert_to_tensor(laplacian, tf.float32)
        graph_signal = inputs

        # Compute ChebNet polynomials
        cheb_polys = self.cheb_polynomials(laplacian, self.k)

        # ChebNet convolution
        cheb_filter = tf.linalg.matrix_transpose(self.kernel)
        cheb_conv = tf.matmul(tf.concat([tf.matmul(graph_signal, cheb_poly) for cheb_poly in cheb_polys], axis=-1), cheb_filter)
        output = self.activation(cheb_conv)

        return output

class SAGEConv(layers.Layer):
    def __init__(self, num_neighbors, num_filters, activation='relu', **kwargs):
        super(SAGEConv, self).__init__(**kwargs)
        self.num_neighbors = num_neighbors
        self.num_filters = num_filters
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        self.kernel = self.add_weight("kernel",
                                      (input_shape[-1] * (self.num_neighbors + 1),
                                       self.num_filters))

    def call(self, inputs, adjacency_matrix):
        adjacency_matrix = tf.cast(adjacency_matrix, tf.float32)
        graph_signal = inputs

        # Aggregate neighbors using mean
        aggregated_neighbors = tf.matmul(adjacency_matrix, graph_signal) / (
                tf.reduce_sum(adjacency_matrix, axis=-1, keepdims=True) + 1e-7)

        # Concatenate aggregated neighbors and input features
        concatenated_features = tf.concat([graph_signal, aggregated_neighbors], axis=-1)

        # GraphSAGE convolution
        output = tf.matmul(concatenated_features, self.kernel)
        output = self.activation(output)

        return output
class GraphNeuralNetwork(tf.keras.Model):
    def __init__(self, ops, embed_dim):
        super(GraphNeuralNetwork, self).__init__()

        # Embedding layer for "op" node set
        self.embedding_layer = _OpEmbedding(ops, embed_dim)

        # ChebNet layer
        self.chebnet_layer = CNConv(num_filters=64, k=2)

        # SAGEConv layer
        self.sageconv_layer = SAGEConv(num_neighbors=3, num_filters=32)

        # Output layer (you may modify this based on your specific task)
        self.output_layer = tf.keras.layers.Dense(units=1, activation='linear')

    def call(self, graph, training=False):
        # Embedding layer
        graph_embedded_ops = self.embedding_layer(graph)

        # ChebNet layer
        config_adj = implicit.AdjacencyMultiplier(graph_embedded_ops, 'config')
        chebnet_output = self.chebnet_layer(graph_embedded_ops.node_sets['nconfig']['feats'], config_adj)

        # SAGEConv layer
        sageconv_output = self.sageconv_layer(chebnet_output, config_adj)

        # Global pooling (you may want to customize this based on your task)
        global_pooled = tf.reduce_mean(sageconv_output, axis=1)

        # Output layer
        model_output = self.output_layer(global_pooled)

        return model_output

In [22]:
gnn_model = GraphNeuralNetwork(ops=ops, embed_dim=16)

In [23]:
gnn_model.compile(optimizer='adam', loss='mse')

In [24]:
#gnn_model.fit(layout_train, validation_data=layout_valid, epochs=10)

In [25]:
import tqdm
_INFERENCE_CONFIGS_BATCH_SIZE = 50

output_csv_filename = f'inference_layout_{SOURCE}_{SEARCH}.csv'
print('\n\n   Running inference on test set ...\n\n')
test_rankings = []

assert layout_npz_dataset.test.graph_id is not None
for graph in tqdm.tqdm(layout_npz_dataset.test.iter_graph_tensors(),
                       total=layout_npz_dataset.test.graph_id.shape[-1],
                       desc='Inference'):
    num_configs = graph.node_sets['g']['runtimes'].shape[-1]
    all_scores = []
    for i in tqdm.tqdm(range(0, num_configs, _INFERENCE_CONFIGS_BATCH_SIZE)):
        end_i = min(i + _INFERENCE_CONFIGS_BATCH_SIZE, num_configs)
        # Take a cut of the configs.
        node_set_g = graph.node_sets['g']
        subconfigs_graph = tfgnn.GraphTensor.from_pieces(
            edge_sets=graph.edge_sets,
            node_sets={
                'op': graph.node_sets['op'],
                'nconfig': tfgnn.NodeSet.from_fields(
                    sizes=graph.node_sets['nconfig'].sizes,
                    features={
                        'feats': graph.node_sets['nconfig']['feats'][:, i:end_i],
                    }),
                'g': tfgnn.NodeSet.from_fields(
                    sizes=tf.constant([1]),
                    features={
                        'graph_id': node_set_g['graph_id'],
                        'runtimes': node_set_g['runtimes'][:, i:end_i],
                        'kept_node_ratio': node_set_g['kept_node_ratio'],
                    })
            })
        h = model.forward(subconfigs_graph, num_configs=(end_i - i),
                          backprop=False)
        all_scores.append(h[0])
    all_scores = tf.concat(all_scores, axis=0)
    graph_id = graph.node_sets['g']['graph_id'][0].numpy().decode()
    sorted_indices = tf.strings.join(
        tf.strings.as_string(tf.argsort(all_scores)), ';').numpy().decode()
    test_rankings.append((graph_id, sorted_indices))

with tf.io.gfile.GFile(output_csv_filename, 'w') as fout:
    fout.write('ID,TopConfigs\n')
    for graph_id, ranks in test_rankings:
        fout.write(f'layout:{SOURCE}:{SEARCH}:{graph_id},{ranks}\n')
print('\n\n   ***  Wrote', output_csv_filename, '\n\n')



   Running inference on test set ...




Inference: 100%|██████████| 8/8 [1:10:47<00:00, 530.91s/it]



   ***  Wrote inference_layout_xla_random.csv 


